In [1]:
import pandas as pd
import numpy as np
from sklearn.utils import shuffle
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense

# Load CSV files
df_one = pd.read_csv("/home/harald_stabbetorp/Master/e-prop_master_nmbu/sound_fet/one_encoded_features.csv")  # Shape: (num_samples, 12800)
df_two = pd.read_csv("/home/harald_stabbetorp/Master/e-prop_master_nmbu/sound_fet/two_encoded_features.csv")  # Shape: (num_samples, 12800)
df_silence = pd.read_csv("/home/harald_stabbetorp/Master/e-prop_master_nmbu/sound_fet/zeros.csv")  # Shape: (num_samples, 12800)

# Reshape to (num_samples, 25, 512)
X_one = df_one.values.reshape(-1, 25, 512)
X_two = df_two.values.reshape(-1, 25, 512)
X_silence = df_silence.values.reshape(-1, 25, 512)

# Define labels: Silence=0, "one"=1, "two"=2
y_silence = np.zeros((X_silence.shape[0], 1))
y_one = np.ones((X_one.shape[0], 1))  # Label "one" as 1
y_two = np.full((X_two.shape[0], 1), 2)  # Label "two" as 2

# Concatenate sequences in alternating order
X_combined = np.vstack((X_silence, X_one, X_silence, X_two))
y_combined = np.vstack((y_silence, y_one, y_silence, y_two))

# Shuffle the dataset (important)
X_combined, y_combined = shuffle(X_combined, y_combined, random_state=42)

# Split into training and testing (using the same data for both)
X_train, X_test = X_combined, X_combined
y_train, y_test = y_combined, y_combined

print("X_train shape:", X_train.shape)  # (total_samples, 25, 512)
print("y_train shape:", y_train.shape)  # (total_samples, 1)


2025-03-12 12:58:35.868073: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1741780715.939225  836072 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1741780715.955956  836072 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-03-12 12:58:36.038920: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


X_train shape: (4, 25, 512)
y_train shape: (4, 1)


In [2]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense

# Define model
inputs = Input(shape=(25, 512))
x = LSTM(256, return_sequences=False)(inputs)
x = Dense(128, activation='relu')(x)
x = Dense(3, activation='softmax')(x)  # 3 output classes: Silence, One, Two

model = Model(inputs, x)
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=20, batch_size=32, validation_split=0.2)


2025-03-12 12:58:42.314493: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:152] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


Epoch 1/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step - accuracy: 0.6667 - loss: 1.0731 - val_accuracy: 0.0000e+00 - val_loss: 1.1051
Epoch 2/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 218ms/step - accuracy: 0.3333 - loss: 0.7180 - val_accuracy: 1.0000 - val_loss: 1.0972
Epoch 3/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 216ms/step - accuracy: 1.0000 - loss: 0.6276 - val_accuracy: 1.0000 - val_loss: 1.0892
Epoch 4/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 212ms/step - accuracy: 1.0000 - loss: 0.4585 - val_accuracy: 1.0000 - val_loss: 1.0821
Epoch 5/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 201ms/step - accuracy: 1.0000 - loss: 0.4129 - val_accuracy: 1.0000 - val_loss: 1.0743
Epoch 6/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 211ms/step - accuracy: 1.0000 - loss: 0.3803 - val_accuracy: 1.0000 - val_loss: 1.0656
Epoch 7/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 207ms/step - accuracy: 1.0000 - loss: 0.3641 - val_accuracy: 1.0000 - val_loss: 1.0556
Epoch 8/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 220ms/step - accuracy: 1.0000 - loss: 0.3561 - val_accuracy: 1.0000 - val_loss

In [3]:
# Evaluate on the test set
test_loss, test_acc = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {test_acc:.4f}")

# Make predictions
y_pred = model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)  # Convert softmax output to class index

print("Predicted Classes:", y_pred_classes[:10])  # Show first 10 predictions
print("True Classes:", y_test.flatten()[:10])  # Show first 10 actual labels


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 104ms/step - accuracy: 1.0000 - loss: 0.0598
Test Accuracy: 1.0000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 277ms/step
Predicted Classes: [1 2 0 0]
True Classes: [1. 2. 0. 0.]
